FOOD ORDERING AI AGENT

Understand the user’s food request
Decide what food fits best
Choose best restaurant
Place the order (simulated)
Return final confirmation

Overview:
Client (Web/App)
   ↓
API Gateway (FastAPI)
   ↓
Auth & Rate Limiting
   ↓
LangGraph Controller
   ↓
Agent A (Planner)
   ↓
Agent B (Executor)
   ↓
Agent C (Verifier)
   ↓
Database / APIs
   ↓
Response



In [7]:
!pip install langgraph langchain langchain-community


In [8]:
from langgraph.graph import StateGraph
from typing import TypedDict, List


In [9]:
#define agent state

class OrderState(TypedDict):
    user_input: str
    menu: list
    selected_item: dict
    final_response: str


In [10]:
#define menu

MENU = [
    {"item": "Veg Burger", "price": 199},
    {"item": "Cheese Pizza", "price": 299},
    {"item": "Pasta", "price": 249},
    {"item": "Veg Biryani", "price": 229}
]


In [11]:
#define agent nodes

#planner node
def planner(state: OrderState):
    return {
        "user_input": state["user_input"],
        "menu": MENU
    }


#selector node
def select_food(state: OrderState):
    # Choose cheapest item
    cheapest = min(state["menu"], key=lambda x: x["price"])
    return {"selected_item": cheapest}


In [12]:
#Final Response Node

def finalize_order(state: OrderState):
    item = state["selected_item"]
    return {
        "final_response": f"""
🍽️ Order Confirmed!

Item: {item['item']}
Price: ₹{item['price']}

Thank you for ordering!
"""
    }


In [13]:
#Build LangGraph

from langgraph.graph import StateGraph

graph = StateGraph(OrderState)

graph.add_node("planner", planner)
graph.add_node("selector", select_food)
graph.add_node("final", finalize_order)

graph.set_entry_point("planner")
graph.add_edge("planner", "selector")
graph.add_edge("selector", "final")

app = graph.compile()


In [14]:
result = app.invoke({"user_input": "I want something cheap to eat"})
print(result["final_response"])



🍽️ Order Confirmed!

Item: Veg Burger
Price: ₹199

Thank you for ordering!

